In [ ]:
import json
import pickle
import re

In [ ]:
data = []
with open("../data/labelled_data_20210311.jsonl", "r", encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))

In [ ]:
new_gold = []
for each in data:
    temp_labels = each.get("labels")
    temp_text = each.get("text")
    
    text_splitted = re.split("\s|!|\.|,|\\(|\\)", temp_text)
    text_splitted = [e for e in text_splitted if len(e) > 1]

    text_splitted = [{e: "O"} for e in text_splitted]
    
    ent_to_put_in = []
    for l in temp_labels:
        ent = temp_text[l[0]:l[1]]
        label = l[2]
        ent_splitted = ent.split()
        # print(ent_splitted)
        for e in ent_splitted:
            temp_e = {e: label}
            ent_to_put_in.append(temp_e)
    
    ent_to_put_in = [i for n, i in enumerate(ent_to_put_in) if i not in ent_to_put_in[n + 1:]]
    # print(ent_to_put_in)
    ent = []
    for i in ent_to_put_in:
        for k,v in i.items():
            ent.append(k)
    
    new_split = []
    
    for each_text in text_splitted:
        for k,v in each_text.items():
            if k in ent: 
                for each_item in ent_to_put_in:
                    for i, o in each_item.items():
                        if k == i:
                            temp_each_text = {i: o}
                            new_split.append(temp_each_text)
            else:
                new_split.append(each_text)
    
    new_gold.append({"text": temp_text, "with_labels": new_split})

In [ ]:
with open("../data/prepared/20210311_eval.pickle", "wb") as fp:
    pickle.dump(new_gold, fp)